<a href="https://colab.research.google.com/github/warakhan/Msic-Generation-with-AI/blob/main/MUSIC_GENERATION_WITH_AI_TASK3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
pip install music21

In [20]:
from google.colab import drive

In [23]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
import os
import numpy as np
import tensorflow as tf
from google.colab import drive
import librosa
from IPython.display import Audio

# Mount Google Drive
drive.mount('/content/drive')

# Path to the audio file
file_path = '/content/drive/My Drive/content/Attention.mp3'

# Load and play original audio file
audio_data, sr = librosa.load(file_path, sr=None)
print("Original audio:")
Audio(data=audio_data, rate=sr)

# Feature extraction
hop_length = 512
n_fft = 2048
n_mfcc = 13

mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc).T

# Normalize MFCCs
mfccs_mean = np.mean(mfccs, axis=0)
mfccs_std = np.std(mfccs, axis=0)
mfccs_normalized = (mfccs - mfccs_mean) / mfccs_std

# Define sequences and targets
sequence_length = 100
X = []
y = []
for i in range(len(mfccs_normalized) - sequence_length):
    X.append(mfccs_normalized[i:i+sequence_length])
    y.append(mfccs_normalized[i+sequence_length])

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Define RNN model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(sequence_length, n_mfcc)),
    tf.keras.layers.Dense(n_mfcc)
])

model.compile(loss='mse', optimizer='adam')

# Train the model
model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2)

# Generate music sequence
seed_sequence = X[0]
generated_sequence = []

for i in range(1000):  # Adjust the length of the generated sequence as needed
    # Predict the next step
    next_step = model.predict(seed_sequence.reshape(1, sequence_length, n_mfcc))
    generated_sequence.append(next_step)
    # Update seed sequence for next prediction
    seed_sequence = np.concatenate((seed_sequence[1:], next_step), axis=0)

# Convert generated sequence back to audio
generated_audio = librosa.feature.inverse.mfcc_to_audio(np.array(generated_sequence).squeeze().T)

# Play the generated audio
print("Generated audio:")
Audio(data=generated_audio, rate=sr)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Original audio:
Epoch 1/50
271/271 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.4322 - val_loss: 0.0991
Epoch 2/50
271/271 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.1165 - val_loss: 0.0807
Epoch 3/50
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.1003 - val_loss: 0.0832
Epoch 4/50
271/271 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0974 - val_loss: 0.0768
Epoch 5/50
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0961 - val_loss: 0.0771
Epoch 6/50
271/271 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0952 - val_loss: 0.0755
Epoch 7/50
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0924 - val_loss: 0.0848
Epoch 8/50
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0927 - val_loss: 0.0794
Epoch 9/50
271/271 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0921 - val_loss: 0.0791
Epoch 10/50
271/271 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0923 - val_loss: 